<a href="https://colab.research.google.com/github/BickNutler/Springboard/blob/main/Case_Study_Country_Club_SQLite_Nicholas_Butler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***NOTE: The code used to complete this assignment was generated through the assistance of ChatGPT.\***

In [6]:
import sqlite3

class LocalSQLConnection:
    def __init__(self, db_name='mydatabase.db'):
        self.db_name = db_name
        self.conn = None
        self.cursor = None

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.conn:
            self.conn.commit()
            self.conn.close()

    def execute_query(self, query, params=()):
        if not self.cursor:
            raise ConnectionError("Database connection not established. Use 'with LocalSQLConnection():' or call 'connect()'.")
        self.cursor.execute(query, params)
        return self.cursor

    def fetch_all(self, query, params=()):
        cursor = self.execute_query(query, params)
        return cursor.fetchall()

    def fetch_one(self, query, params=()):
        cursor = self.execute_query(query, params)
        return cursor.fetchone()

    def create_table(self, table_name, columns):
        # columns should be a dictionary like {'column_name': 'data_type'}
        column_definitions = ', '.join([f'{name} {dtype}' for name, dtype in columns.items()])
        query = f'CREATE TABLE IF NOT EXISTS {table_name} ({column_definitions})'
        self.execute_query(query)

    def insert_data(self, table_name, data):
        # data should be a dictionary like {'column_name': 'value'}
        columns = ', '.join(data.keys())
        placeholders = ', '.join(['?'] * len(data))
        query = f'INSERT INTO {table_name} ({columns}) VALUES ({placeholders})'
        self.execute_query(query, tuple(data.values()))

    def update_data(self, table_name, data, condition):
        # data should be a dictionary like {'column_name': 'value'}
        set_clause = ', '.join([f'{name} = ?' for name in data.keys()])
        query = f'UPDATE {table_name} SET {set_clause} WHERE {condition}'
        self.execute_query(query, tuple(data.values()))

    def delete_data(self, table_name, condition):
        query = f'DELETE FROM {table_name} WHERE {condition}'
        self.execute_query(query)

    def connect(self, db_name=None):
        if db_name:
            self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()

    def close(self):
        if self.conn:
            self.conn.commit()
            self.conn.close()
            self.conn = None
            self.cursor = None

with LocalSQLConnection('/content/sqlite_db_pythonsqlite.db') as db:
    db.execute_query("SELECT name FROM sqlite_master WHERE type='table';")
    tables = db.fetch_all("SELECT name FROM sqlite_master WHERE type='table';")
    print("Tables in the database:")
    for table in tables:
        print(table[0])

Tables in the database:
Bookings
Facilities
Members


In [7]:
import os

db_path = '/content/sqlite_db_pythonsqlite.db'
print("Database exists:", os.path.exists(db_path))
print("File size:", os.path.getsize(db_path) if os.path.exists(db_path) else "File not found")


Database exists: True
File size: 217088


The data you need is in the "country_club" database. This database
contains 3 tables:
i) the "Bookings" table,
ii) the "Facilities" table, and
iii) the "Members" table.

In this case study, you'll be asked a series of questions. You can
solve them using the platform, but for the final deliverable,
paste the code for each solution into this script, and upload it
to your GitHub.

Before starting with the questions, feel free to take your time,
exploring the data, and getting acquainted with the 3 tables.

/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine.

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it.

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to.

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.

QUESTIONS:


**Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!**

In [8]:
query = """
SELECT
    f.name AS facility_name,
    SUM(
        CASE
            WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
        END
    ) AS total_revenue
FROM
    Bookings b
JOIN
    Facilities f ON b.facid = f.facid
GROUP BY
    f.name
HAVING
    total_revenue < 1000
ORDER BY
    total_revenue;
"""

with LocalSQLConnection('/content/sqlite_db_pythonsqlite.db') as db:
    results = db.fetch_all(query)

# Convert column headers + results into one list for width calculations
headers = ["Facility Name", "Total Revenue"]
rows = [(row[0], f"{row[1]:.2f}") for row in results]
all_rows = [headers] + rows

# Calculate max width of each column
col_widths = [max(len(str(r[i])) for r in all_rows) for i in range(len(headers))]

# Helper to format a single row
def format_row(row):
    return "| " + " | ".join(f"{str(val):<{col_widths[i]}}" for i, val in enumerate(row)) + " |"

# Print the table
print(format_row(headers))
print("|-" + "-|-".join("-" * w for w in col_widths) + "-|")
for row in rows:
    print(format_row(row))



| Facility Name | Total Revenue |
|---------------|---------------|
| Table Tennis  | 180.00        |
| Snooker Table | 240.00        |
| Pool Table    | 270.00        |


**Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order**

In [9]:
query = """
SELECT
    m.surname || ', ' || m.firstname AS member_name,
    COALESCE(r.surname || ', ' || r.firstname, 'None') AS recommended_by
FROM
    Members m
LEFT JOIN
    Members r ON m.recommendedby = r.memid
ORDER BY
    m.surname,
    m.firstname;
"""

with LocalSQLConnection('sqlite_db_pythonsqlite.db') as db:
    results = db.fetch_all(query)

    # Format and print nicely
    headers = ["Member Name", "Recommended By"]
    rows = [(row[0], row[1]) for row in results]
    all_rows = [headers] + rows
    col_widths = [max(len(str(r[i])) for r in all_rows) for i in range(len(headers))]

    def format_row(row):
        return "| " + " | ".join(f"{str(val):<{col_widths[i]}}" for i, val in enumerate(row)) + " |"

    print(format_row(headers))
    print("|-" + "-|-".join("-" * w for w in col_widths) + "-|")
    for row in rows:
        print(format_row(row))


| Member Name              | Recommended By     |
|--------------------------|--------------------|
| Bader, Florence          | Stibbons, Ponder   |
| Baker, Anne              | Stibbons, Ponder   |
| Baker, Timothy           | Farrell, Jemima    |
| Boothe, Tim              | Rownam, Tim        |
| Butters, Gerald          | Smith, Darren      |
| Coplin, Joan             | Baker, Timothy     |
| Crumpet, Erica           | Smith, Tracy       |
| Dare, Nancy              | Joplette, Janice   |
| Farrell, David           | None               |
| Farrell, Jemima          | None               |
| GUEST, GUEST             | None               |
| Genting, Matthew         | Butters, Gerald    |
| Hunt, John               | Purview, Millicent |
| Jones, David             | Joplette, Janice   |
| Jones, Douglas           | Jones, David       |
| Joplette, Janice         | Smith, Darren      |
| Mackenzie, Anna          | Smith, Darren      |
| Owen, Charles            | Smith, Darren      |


**Q12: Find the facilities with their usage by member, but not guests**

In [10]:
query = """
SELECT
    f.name AS facility_name,
    m.firstname || ' ' || m.surname AS member_name,
    SUM(b.slots) AS total_usage
FROM
    Bookings b
JOIN
    Facilities f ON b.facid = f.facid
JOIN
    Members m ON b.memid = m.memid
WHERE
    b.memid <> 0
GROUP BY
    f.name,
    m.firstname,
    m.surname
ORDER BY
    f.name,
    m.surname,
    m.firstname;
"""

with LocalSQLConnection('sqlite_db_pythonsqlite.db') as db:
    results = db.fetch_all(query)

    # Format for pretty Markdown output
    headers = ["Facility Name", "Member Name", "Total Usage"]
    rows = [(row[0], row[1], row[2]) for row in results]
    all_rows = [headers] + rows
    col_widths = [max(len(str(r[i])) for r in all_rows) for i in range(len(headers))]

    def format_row(row):
        return "| " + " | ".join(f"{str(val):<{col_widths[i]}}" for i, val in enumerate(row)) + " |"

    print(format_row(headers))
    print("|-" + "-|-".join("-" * w for w in col_widths) + "-|")
    for row in rows:
        print(format_row(row))


| Facility Name   | Member Name             | Total Usage |
|-----------------|-------------------------|-------------|
| Badminton Court | Florence Bader          | 27          |
| Badminton Court | Anne Baker              | 30          |
| Badminton Court | Timothy Baker           | 21          |
| Badminton Court | Tim Boothe              | 36          |
| Badminton Court | Gerald Butters          | 63          |
| Badminton Court | Erica Crumpet           | 6           |
| Badminton Court | Nancy Dare              | 30          |
| Badminton Court | Jemima Farrell          | 21          |
| Badminton Court | John Hunt               | 6           |
| Badminton Court | David Jones             | 24          |
| Badminton Court | Douglas Jones           | 6           |
| Badminton Court | Anna Mackenzie          | 96          |
| Badminton Court | Charles Owen            | 18          |
| Badminton Court | David Pinker            | 21          |
| Badminton Court | Millicent Purview   

**Q13: Find the facilities usage by month, but not guests**

In [12]:
query = """
SELECT
    f.name AS facility_name,
    strftime('%Y-%m', b.starttime) AS month,
    SUM(b.slots) AS total_usage
FROM
    Bookings b
JOIN
    Facilities f ON b.facid = f.facid
WHERE
    b.memid <> 0
GROUP BY
    f.name,
    strftime('%Y-%m', b.starttime)
ORDER BY
    f.name,
    month;
"""

with LocalSQLConnection('sqlite_db_pythonsqlite.db') as db:
    results = db.fetch_all(query)

    # Prepare headers and compute column widths
    headers = ["Facility Name", "Month", "Total Usage"]
    rows = [(row[0], row[1], row[2]) for row in results]
    all_rows = [headers] + rows
    col_widths = [max(len(str(r[i])) for r in all_rows) for i in range(len(headers))]

    def format_row(row):
        return "| " + " | ".join(f"{str(val):<{col_widths[i]}}" for i, val in enumerate(row)) + " |"

    # Print Markdown-style table
    print(format_row(headers))
    print("|-" + "-|-".join("-" * w for w in col_widths) + "-|")
    for row in rows:
        print(format_row(row))


| Facility Name   | Month   | Total Usage |
|-----------------|---------|-------------|
| Badminton Court | 2012-07 | 165         |
| Badminton Court | 2012-08 | 414         |
| Badminton Court | 2012-09 | 507         |
| Massage Room 1  | 2012-07 | 166         |
| Massage Room 1  | 2012-08 | 316         |
| Massage Room 1  | 2012-09 | 402         |
| Massage Room 2  | 2012-07 | 8           |
| Massage Room 2  | 2012-08 | 18          |
| Massage Room 2  | 2012-09 | 28          |
| Pool Table      | 2012-07 | 110         |
| Pool Table      | 2012-08 | 303         |
| Pool Table      | 2012-09 | 443         |
| Snooker Table   | 2012-07 | 140         |
| Snooker Table   | 2012-08 | 316         |
| Snooker Table   | 2012-09 | 404         |
| Squash Court    | 2012-07 | 50          |
| Squash Court    | 2012-08 | 184         |
| Squash Court    | 2012-09 | 184         |
| Table Tennis    | 2012-07 | 98          |
| Table Tennis    | 2012-08 | 296         |
| Table Tennis    | 2012-09 | 40

***NOTE: The code used to complete this assignment was generated through the assistance of ChatGPT.\***